#### Work Group 4:
#### Integrantes:
#### Ascencios, Seidy - 20191622
#### Morales, Luana - 2019120
#### Oré, Flavia - 20191215
#### Quintero, Marcela - 20191445


In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import t # t - student 
import os
!pip install pyreadr
import pyreadr as pyreadr

Crear una Clase OLS con las siguientes características:
Los atributos:
DataFrame de variables explicativas.
El vector de la variable Y
Una lista que permita seleccionar las variables del DataFrame (puede ser posiciones o no nombre de variables)
Una variable booleana para errores estándar robustas

In [2]:
# Cargamos la base de datos: 
user = os.getlogin()   # Username


os.chdir(f"C:/Users/{user}/Documents/GitHub/1ECO35_2022_2/Lab4") 

cps2012_env = pyreadr.read_r("../data/cps2012.Rdata") 

# Y debemos extraer la información de la base de datos del diccionario:
cps2012 = cps2012_dict['data']
dt = cps2012.describe()

In [3]:
# Así, la base de datos es:
print(cps2012)

         year       lnw  female  widowed  divorced  separated  nevermarried  \
0      2012.0  1.909543     1.0      0.0       0.0        0.0           0.0   
1      2012.0  1.365773     1.0      0.0       0.0        0.0           0.0   
2      2012.0  2.540223     0.0      0.0       0.0        0.0           0.0   
3      2012.0  1.801091     1.0      0.0       0.0        0.0           0.0   
4      2012.0  3.349904     0.0      0.0       0.0        0.0           0.0   
...       ...       ...     ...      ...       ...        ...           ...   
29212  2012.0  3.978513     0.0      0.0       0.0        0.0           1.0   
29213  2012.0  3.142265     1.0      0.0       0.0        0.0           1.0   
29214  2012.0  2.725619     1.0      0.0       0.0        0.0           0.0   
29215  2012.0  3.142265     0.0      0.0       0.0        0.0           0.0   
29216  2012.0  2.433613     1.0      0.0       0.0        0.0           0.0   

       hsd08  hsd911  hsg  ...   so   we  exp1  exp

In [23]:
# Creamos la clase OLS:

class OLS_G4:
    
    def __init__(self, X,Y):
        
        self.X = X
        self.Y = Y
        
    def coeficientes(self):
        
        self.n = self.X.shape[0] # numero de observaciones, # self.n "Se crea un nuevo atributo"
        k = self.X.shape[1]
        X1 = np.column_stack((np.ones(self.n ), self.X.to_numpy() ))  # self.X.to_numpy()  # DataFrame to numpy
        Y1 = self.Y.to_numpy().reshape(self.n  ,1)  #reshape(-1  ,1)
        self.X1 = X1
        self.Y1 = Y1
        self.beta = np.linalg.inv(X1.T @ X1) @ ((X1.T) @ Y1 )
        self.nk = self.n - k 
        self.Y_est =  self.X1 @ self.beta
        
           
    def estandar(self):
    
        self.coeficientes()
        
        y_est =  self.X1 @ self.beta
        sigma =  sum(list( map( lambda x: x**2 , self.Y1 - y_est)   )) / self.nk 
        Var1 = sigma*np.linalg.inv(self.X1.T @ self.X1)
        self.sd1 = np.sqrt( np.diag(Var1) )
        self.lower_bound1 = self.beta-1.96*self.sd1
        self.upper_bound1 = self.beta+1.96*self.sd1
        
    def robust(self):
    
        self.coeficientes()
        
        y_est =  self.X1 @ self.beta
        matrix_robust = np.diag(list( map( lambda x: x**2 , self.Y1 - y_est)))
        Var2 = np.linalg.inv(self.X1.T @ self.X1) @ self.X1.T @ matrix_robust @ self.X1 @ np.linalg.inv(self.X1.T @ self.X1)
        sd2 = np.sqrt( np.diag(Var2) )
        lower_bound2 = self.beta-1.96*sd2
        upper_bound2 = self.beta+1.96*sd2

    def R2_RMSE(self):
            
         self.coeficientes() # run function 
         
         y_est =  self.X1 @ self.beta
         error = self.Y1 - y_est
         self.SCR = np.sum(np.square(error))
         SCT = np.sum(np.square(self.Y1 - np.mean(self.Y1))) 
         self.R2 = 1 - self.SCR/SCT
         self.rmse = (self.SCR/self.n)**0.5
        
  def Table(self,**Kargs):
        
        self.R2_RMSE()
        self.robust
        self.estándar()
        self.coeficientes()
        scr = self.SCR
        sigma =  scr / self.nk
        Var = sigma*np.linalg.inv(self.X1.T @ self.X1)
        sd = np.sqrt( np.diag(Var) )
        t_est = np.absolute(self.beta/sd)
        
        if (Kargs['Output'] == "DataFrame"):
        
           df = pd.DataFrame( {"coeficientes": self.beta.flatten()  , "error-estandar" : self.sd1.flatten(), "límite-superior": self.upper_bound1.flatten(), "límite-inferior": self.lower_bound1.flatten() } )
                              
           
        elif (Kargs['Output'] == "Diccionario"):
            
           df = {"R^2": self.R2.flatten() ,"Root-MSE": self.rmse.flatten()}
           
        
        return df
                                  
            
            

In [27]:
#flatten():  De multi array a simple array 
cps2012.shape

variance_cols = cps2012.var().to_numpy() # to numpy

Dataset = cps2012.iloc[ : ,  np.where( variance_cols != 0   )[0] ]

X = Dataset.iloc[:,1:3]
Y = Dataset[['lnw']]

In [36]:
Reg1 = OLS_G4(X,Y)
Reg1.X

Reg1.coeficientes()
Reg1.beta

Reg1.R2_RMSE()
Reg1.R2

Reg1.R2_RMSE()
Reg1.rmse 

0.6495008509803024

In [34]:
Reg1.Table(Output = "DataFrame")
Reg1.Table(Output = "R2_result")

[[ 2.9094181 ]
 [-0.25887621]
 [-0.17751519]]
